In [ ]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

root_dir = Path().absolute()
# Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
if root_dir.parts[-1:] == ('notebooks',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
if root_dir.parts[-1:] == ('ccfraud',):
    root_dir = Path(*root_dir.parts[:-1])
    sys.path.append(str(root_dir))
root_dir = str(root_dir) 

print(f"Root dir: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

In [ ]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()

In [ ]:
merchant_fg = fs.get_feature_group("merchant_details", version=1)
account_fg = fs.get_feature_group("account_details", version=1)
bank_fg = fs.get_feature_group("bank_details", version=1)
card_fg = fs.get_feature_group("card_details", version=1)
cc_trans_aggs_fg = fs.get_feature_group("cc_trans_aggs_fg", version=1)
cc_trans_fg = fs.get_feature_group("cc_trans_fg", version=1)

In [ ]:
subtree1 = cc_trans_aggs_fg.select_all()\
    .join(account_fg.select(['debt_end_prev_month']), on="account_id", join_type="inner")\
    .join(bank_fg.select(['credit_rating', 'days_since_bank_cr_changed', 'country']), on="bank_id", join_type="inner")

In [ ]:
# df = subtree1.read()
# df

In [ ]:
selection = cc_trans_fg.select_except(['t_id', 'cc_num', 'merchant_id', 'account_id', 'ip_address', 'ts'])\
    .join(merchant_fg.select_features(), prefix="merchant_", on="merchant_id")\
    .join(subtree1, on="cc_num")

In [ ]:
df = selection.read()
df

In [ ]:
fv = fs.get_or_create_feature_view(name="cc_fraud_fv", 
                                   version=1, 
                                   description="features for a credit card fraud prediction model",
                                   query=selection,
                                   labels=['is_fraud']
                                  )

In [ ]:
test_start="2025-09-15 00:00"
X_train, X_test, y_train, y_test = fv.train_test_split(test_start=test_start)
X_train

In [ ]:
# features, labels = fv2.training_data()
# features.info()

In [ ]:
# version, job = fv2.create_training_data(data_format='parquet')

In [ ]:
# X_train, X_test= fv2.get_training_data(version)